In [1]:
using Gurobi
using JuMP

In [2]:
model = Model(Gurobi.Optimizer)
@variable(model, x <= 10.5, Int)
@objective(model, Max, x)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-01


x

In [3]:
function heur_callback(cb_data)
    x_val = callback_value(cb_data, x)
    status = MOI.submit(
        model, MOI.HeuristicSolution(cb_data), [x], [floor(Int, x_val)]
    )
    println("I submitted a heuristic solution, and the status was: ", status)
end

heur_callback (generic function with 1 method)

In [4]:
cut_called = false
function cut_callback(cb_data)
    cut_called = true
    x_val = callback_value(cb_data, x)
    con = @build_constraint(x <= floor(x_val))
    MOI.submit(model, MOI.UserCut(cb_data), con)
end


cut_callback (generic function with 1 method)

In [5]:
function lazy_callback(cb_data)
    status = callback_node_status(cb_data, model)
    if status == MOI.CALLBACK_NODE_STATUS_FRACTIONAL
        # `callback_value(cb_data, x)` is not integer (to some tolerance).
        # If, for example, your lazy constraint generator requires an
        # integer-feasible primal solution, you can add a `return` here.
        return
    elseif status == MOI.CALLBACK_NODE_STATUS_INTEGER
        # `callback_value(cb_data, x)` is integer (to some tolerance).
    else
        @assert status == MOI.CALLBACK_NODE_STATUS_UNKNOWN
        # `callback_value(cb_data, x)` might be fractional or integer.
    end
    x_val = callback_value(cb_data, x)
    if x_val > 2 + 1e-6
        con = @build_constraint(x <= 2)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
end

lazy_callback (generic function with 1 method)

In [6]:
function lazy1_callback(cb_data)
    if callback_value(x) > 2
        con = @build_constraint(x <= 2)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
end

lazy1_callback (generic function with 1 method)

In [8]:
MOI.set(model, MOI.HeuristicCallback(), heur_callback)
#MOI.set(model, MOI.UserCutCallback(), cut_callback)
#MOI.set(model, MOI.LazyConstraintCallback(), lazy_callback)
#MOI.set(model, MOI.LazyConstraintCallback(), callback_lazy1)

In [9]:
status = optimize!(model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0x04ceb2e1
Variable types: 0 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%

User-callback calls 86, time in user-callback 0.03 sec


In [13]:
using JuMP  # JuMP v.0.20.3, aka the master branch as of 2020-02.07 (and not the latest release)
using Gurobi
using Random
using Test

function foo()
    # Prepare JuMP model
    model = Model(Gurobi.Optimizer)
    @variable(model, 0 <=pgen1 <= 100)
    @variable(model, 0 <=pgen2 <= 100)
    #@constraint(m, pgen1+pgen2==100)
    @objective(model, Max, 5*pgen1+1.2*pgen2)
    lazy_called = false
    function my_callback_function(cb_data)  # callback for preventive constraints
        lazy_called = true
        print("Entered callback function. ")
        TOL = 1e-6
        con = @build_constraint(pgen1 <= 80)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end

    # Set lazy constraint, run optimization and perform tests
    MOI.set(model, MOI.LazyConstraintCallback(), my_callback_function)
    optimize!(model)
    
    #@test termination_status(model) == MOI.OPTIMAL
    #@test primal_status(model) == MOI.FEASIBLE_POINT
    #@test lazy_called
end

foo()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-01
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x009f5141
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.2000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.200000000e+02

User-callback calls 24, time in user-callback 0.00 sec
